In [1]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
pd.options.display.float_format = '{:.2f}'.format

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#reading data
dfv = pd.read_csv('dfV_original.csv')
dfc = pd.read_csv('dfC_orignial.csv')
#create a new column based on the value of Platform variable
dfv['MediaType'] = dfv['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')
dfc['MediaType'] = dfc['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')

In [3]:
#create a dataset without duplicates
dfv_Dupeless = dfv.dropna(subset=['Graphics'])
dfv_Dupeless

,Case,Platform,Publisher,VideoURL,PubDate,Likes/Hearts,Comments,Saves,Shots,Graphics,...,Sourcing,Fairness,Agenda,Media Intergrity,Decorum,Language,Topic,Presentation,ExternalVidSource,MediaType
0,1,YouTube Shorts,The Washington Post,This high school student is working to unify A...,11/1/23,100,11,NaN,1.00,3.00,...,0.00,4.00,3.00,3.00,2.00,2.00,2.00,3.00,0.00,Hybrid
1,2,YouTube Shorts,The Washington Post,Visit one of the tallest timber buildings in t...,11/1/23,26,2,NaN,1.00,4.00,...,1.00,2.00,4.00,1.00,3.00,2.00,7.00,3.00,0.00,Hybrid
2,3,YouTube Shorts,The Washington Post,(5) Rep. George Santos survives vote to expel ...,11/1/23,278,70,NaN,1.00,1.00,...,1.00,2.00,4.00,2.00,2.00,1.00,2.00,2.00,0.00,Hybrid
3,4,YouTube Shorts,The Washington Post,Trump testifies in his own civil fraud trial (...,11/6/23,783,193,NaN,1.00,1.00,...,1.00,2.00,3.00,2.00,99.00,2.00,2.00,3.00,0.00,Hybrid
4,5,YouTube Shorts,The Washington Post,Abortion on the ballot gives Dems a big night ...,11/8/23,277,75,NaN,1.00,1.00,...,1.00,2.00,3.00,2.00,1.00,1.00,2.00,2.00,0.00,Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,122,TikTok,GoodMorning BadNews,#goodmorningbadnews #initedstates #israel #pal...,10/25/23,105700,1856,11400.00,1.00,3.00,...,1.00,3.00,2.00,3.00,99.00,4.00,2.00,4.00,0.00,SMonly
122,123,TikTok,GoodMorning BadNews,#goodmorningbadnews #military #usmilitary #occ...,11/11/23,43400,1297,4033.00,1.00,2.00,...,1.00,4.00,1.00,5.00,4.00,5.00,2.00,5.00,0.00,SMonly
123,124,TikTok,GoodMorning BadNews,OUR FAVORITE KIND OF UPDATE! #henrykissinger #...,11/30/23,103500,810,9535.00,1.00,3.00,...,1.00,2.00,1.00,3.00,4.00,5.00,2.00,5.00,0.00,SMonly
124,125,TikTok,GoodMorning BadNews,#goodmorningbadnews #money #pay #paycheck #pri...,1/1/24,115500,1855,11800.00,1.00,2.00,...,1.00,3.00,2.00,3.00,4.00,4.00,6.00,4.00,0.00,SMonly


In [4]:
#filling uncoded duplicate videos and checking if it's done correctly
dfv_dupefill=dfv.copy()
dfv_dupefill=dfv_dupefill.groupby(['VideoURL']).bfill().ffill()
dfv_dupefill['VideoURL'] = dfv['VideoURL']
dfv_dupefill.to_csv('dfV_dupefill.csv', index=False)

In [5]:
#if above is correct, overwriting the original dfv with the new dfv_dupefill
dfv=dfv_dupefill

In [6]:
#change values with 99 into NaN
dfv = dfv.replace(99, np.nan)

In [7]:

#reverse coded variables
dfv2 = dfv.copy()
dfv2['Fairness'] = 6-dfv.Fairness
dfv2['Agenda'] = 6-dfv['Agenda']
dfv2['Media Intergrity'] = 6-dfv['Media Intergrity']
#add Animation variable based on values in dfv. if Graphics is 1, then Animation is 1, otherwise 
# dfv2['Graphics'] = dfv['Graphics']
# dfv2['Animation'] = dfv2['Graphics'].apply(lambda x: 1 if x == 1 else 0)
# dfv2['Graphics'] = dfv['Graphics']
# dfv2['Filters'] = dfv2['Graphics'].apply(lambda x: 1 if x == 2 else 0)
# dfv['Graphics'] = dfv['Graphics']
dfv2['Animation'] = np.where(dfv['Graphics']==1, 1, 0)
dfv2['Filters'] = np.where(dfv['Graphics']==2, 1, 0)

#creating a 0 to 1 variable for some variables
dfv2['Decorum_Z']=(dfv['Decorum']-1)/4
dfv2['Language_Z']=(dfv['Language']-1)/4

dfv_Dupeless2 = dfv_Dupeless.copy()
dfv_Dupeless2['Fairness'] = 6 - dfv_Dupeless['Fairness']
dfv_Dupeless2['Agenda'] = 6 - dfv_Dupeless['Agenda']
dfv_Dupeless2['Media Intergrity'] = 6 - dfv_Dupeless['Media Intergrity']

#add Animation variable based on values in dfv. if Graphics is 1, then Animation is 1, otherwise 0
# dfv_Dupeless2['Graphics'] = dfv_Dupeless['Graphics']
# dfv_Dupeless2['Animation'] = dfv_Dupeless2['Graphics'].apply(lambda x: 1 if x == 1 else 0)
# dfv_Dupeless2['Graphics'] = dfv_Dupeless['Graphics']
# dfv_Dupeless2['Filters'] = dfv_Dupeless2['Graphics'].apply(lambda x: 1 if x == 2 else 0)
# dfv_Dupeless2['Graphics'] = dfv_Dupeless['Graphics']
dfv_Dupeless2['Animation'] = np.where(dfv_Dupeless['Graphics']==1, 1, 0)
dfv_Dupeless2['Filters'] = np.where(dfv_Dupeless['Graphics']==2, 1, 0)
#creating a 0 to 1 variable for some variables
dfv_Dupeless2['Decorum_Z']=(dfv_Dupeless['Decorum']-1)/4
dfv_Dupeless2['Language_Z']=(dfv_Dupeless['Language']-1)/4

#overiting the original datasets
dfv=dfv2
dfv_Dupeless=dfv_Dupeless2

dfv_Dupeless.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv_Dupeless.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfv.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfc.rename(columns={'Likes/Hearts': 'CommLikes'}, inplace=True)
dfc.rename(columns={'Comment': 'CommentText'}, inplace=True)

In [8]:
dfv_Dupeless.Graphics.value_counts()

Graphics
1.00    36
4.00    30
3.00    20
2.00    14
Name: count, dtype: int64

In [9]:
#recode variables with value labels
dfv_recode = dfv.copy()
dfv_recode['Shots'] = dfv['Shots'].replace({1: 'steady', 0: 'not steady', 99: np.nan})
dfv_recode['Graphics'] = dfv['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: np.nan})
dfv_recode['EditStyle'] = dfv['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: np.nan})
dfv_recode['Broll'] = dfv['Broll'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['NatSound'] = dfv['NatSound'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['Audio'] = dfv['Audio'].replace({1: 'good', 0: 'poor', 99: np.nan})
dfv_recode['Music'] = dfv['Music'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_recode['Narration'] = dfv['Narration'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_recode['Humor'] = dfv['Humor'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['Offensive'] = dfv['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: np.nan})
dfv_recode['Sourcing'] = dfv['Sourcing'].replace({1: 'use', 0: 'not use', 99: np.nan})
dfv_recode['Topic'] = dfv['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})

dfv_Dupeless_recode = dfv_Dupeless.copy()
dfv_Dupeless_recode['Shots'] = dfv_Dupeless['Shots'].replace({1: 'steady', 0: 'not steady', 99: np.nan})
dfv_Dupeless_recode['Graphics'] = dfv_Dupeless['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: np.nan})
dfv_Dupeless_recode['EditStyle'] = dfv_Dupeless['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: np.nan})
dfv_Dupeless_recode['Broll'] = dfv_Dupeless['Broll'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['NatSound'] = dfv_Dupeless['NatSound'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['Audio'] = dfv_Dupeless['Audio'].replace({1: 'good', 0: 'poor', 99: np.nan})
dfv_Dupeless_recode['Music'] = dfv_Dupeless['Music'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_Dupeless_recode['Narration'] = dfv_Dupeless['Narration'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_Dupeless_recode['Humor'] = dfv_Dupeless['Humor'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['Offensive'] = dfv_Dupeless['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: np.nan})
dfv_Dupeless_recode['Sourcing'] = dfv_Dupeless['Sourcing'].replace({1: 'use', 0: 'not use', 99: np.nan})
dfv_Dupeless_recode['Topic'] = dfv_Dupeless['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})

In [10]:
dfv_ColNames = dfv_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_ColNames.loc[:,i] = dfv_ColNames[i].astype(str) + ' ({})'.format(i)

dfv_Dupeless_ColNames = dfv_Dupeless_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_Dupeless_ColNames.loc[:,i] = dfv_Dupeless_ColNames[i].astype(str) + ' ({})'.format(i)

In [11]:
#recade variables with value labels
# dfc['Focus'] = dfc['Focus'].replace({1: 'central',  0: 'peripheral', 99: 'NA'})
# change 99 to NA in dfc
dfc = dfc.replace({99: np.nan})
# change -93 to NA in dfv
dfv = dfv.replace({-93: np.nan})
dfv = dfv.replace({99: np.nan})
dfv_Dupeless = dfv_Dupeless.replace({99: np.nan})
dfv_Dupeless = dfv_Dupeless.replace({-93: np.nan})


In [12]:
#saving files
dfv.to_csv('Analysis/dfv.csv', index=False)
dfv_Dupeless.to_csv('Analysis/dfv_noDupe.csv', index=False)
dfv_recode.to_csv('Analysis/dfv_recode.csv', index=False)
dfv_Dupeless_recode.to_csv('Analysis/dfv_noDupe_recode.csv', index=False)
dfv_ColNames.to_csv('Analysis/dfv_ColNames.csv', index=False)
dfv_Dupeless_ColNames.to_csv('Analysis/dfv_noDupe_ColNames.csv', index=False)
dfc.to_csv('Analysis/dfc.csv', index=False)

In [13]:

IVlist = ['Platform', 'Publisher', 'MediaType']
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']

For each DV, define which value is Trendy (1) and which value is Traditional (0).
Add up the variables to create a Composite variable, Trendiness.
You dont have to have all the DVs for the composite variable.
But they have to be 3 or more.

Cronbach's Alpha.
Test for reliability of composite variables
>.80, reliable

In [14]:
# # Testing different between Publisher
# # Specify the subfolder path
# subfolder_path = 'Publisher/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [15]:
# # Testing different between Platform
# # Specify the subfolder path
# subfolder_path = 'Platform/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [16]:
# # Testing different between MediaType
# # Specify the subfolder path
# subfolder_path = 'MediaType/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i + ": " + "chi2 = " + chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [17]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_Dupeless_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,MediaType,2,97,10.93,0.00,0.18
1,Comments,MediaType,2,97,2.69,0.07,0.05
2,MediaIntegrity,MediaType,2,97,12.48,0.00,0.20
3,Presentation,MediaType,2,97,60.28,0.00,0.55


In [18]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,MediaType,Hybrid,Legacy,False,True,0.25,64.33,two-sided,0.80,1.00,bonf,0.257,0.06
1,Likes,MediaType,Hybrid,SMonly,False,True,-3.23,32.51,two-sided,0.00,0.01,bonf,17.872,-0.83
2,Likes,MediaType,Legacy,SMonly,False,True,-3.28,32.62,two-sided,0.00,0.01,bonf,19.693,-0.80
3,Comments,MediaType,Hybrid,Legacy,False,True,-1.28,45.81,two-sided,0.21,0.62,bonf,0.502,-0.33
4,Comments,MediaType,Hybrid,SMonly,False,True,-2.28,45.42,two-sided,0.03,0.08,bonf,2.173,-0.57
5,Comments,MediaType,Legacy,SMonly,False,True,-0.92,59.61,two-sided,0.36,1.00,bonf,0.37,-0.23
6,MediaIntegrity,MediaType,Hybrid,Legacy,False,True,-1.50,63.98,two-sided,0.14,0.42,bonf,0.645,-0.36
7,MediaIntegrity,MediaType,Hybrid,SMonly,False,True,3.50,63.10,two-sided,0.00,0.00,bonf,35.914,0.84
8,MediaIntegrity,MediaType,Legacy,SMonly,False,True,4.77,59.38,two-sided,0.00,0.00,bonf,1428.771,1.19
9,Presentation,MediaType,Hybrid,Legacy,False,True,4.06,64.96,two-sided,0.00,0.00,bonf,173.068,0.97


In [19]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Hybrid,5783.03,8603.68
1,Likes,Legacy,5281.07,7947.59
2,Likes,SMonly,35076.97,50672.91
3,Comments,Hybrid,237.92,528.39
4,Comments,Legacy,466.07,856.32
5,Comments,SMonly,682.12,991.54
6,MediaIntegrity,Hybrid,3.55,0.80
7,MediaIntegrity,Legacy,3.83,0.75
8,MediaIntegrity,SMonly,2.84,0.88
9,Presentation,Hybrid,2.95,0.84


In [20]:
# # Define the IV and DV variables
# IV = 'Publisher'
# DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
# DF = dfv_Dupeless_ColNames
# # Run the ANOVA
# for i in DV:
#     anova_result = pg.anova(data=DF, dv=i, between=IV)
#     globals()[f"ANOVA_{i}"] = anova_result
#     globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
#     pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
#     globals()[f"Pairwise_{i}"] = pairwise_comparisons
#     globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
#     mean_table = DF.groupby(IV)[i].mean().reset_index()
#     std_table = DF.groupby(IV)[i].std().reset_index()
#     mean_std_table = pd.merge(mean_table, std_table, on=IV)
#     mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
#     mean_std_table = mean_std_table.rename(columns={i: 'Value'})
#     globals()[f"mean_std_table_{i}"] = mean_std_table
#     globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
# merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
# merged_anova.reset_index(drop=True, inplace=True)
# merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
# merged_pairwise.reset_index(drop=True, inplace=True)
# merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
# merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
# merged_anova

In [21]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,MediaType,Hybrid,Legacy,False,True,0.25,64.33,two-sided,0.80,1.00,bonf,0.257,0.06
1,Likes,MediaType,Hybrid,SMonly,False,True,-3.23,32.51,two-sided,0.00,0.01,bonf,17.872,-0.83
2,Likes,MediaType,Legacy,SMonly,False,True,-3.28,32.62,two-sided,0.00,0.01,bonf,19.693,-0.80
3,Comments,MediaType,Hybrid,Legacy,False,True,-1.28,45.81,two-sided,0.21,0.62,bonf,0.502,-0.33
4,Comments,MediaType,Hybrid,SMonly,False,True,-2.28,45.42,two-sided,0.03,0.08,bonf,2.173,-0.57
5,Comments,MediaType,Legacy,SMonly,False,True,-0.92,59.61,two-sided,0.36,1.00,bonf,0.37,-0.23
6,MediaIntegrity,MediaType,Hybrid,Legacy,False,True,-1.50,63.98,two-sided,0.14,0.42,bonf,0.645,-0.36
7,MediaIntegrity,MediaType,Hybrid,SMonly,False,True,3.50,63.10,two-sided,0.00,0.00,bonf,35.914,0.84
8,MediaIntegrity,MediaType,Legacy,SMonly,False,True,4.77,59.38,two-sided,0.00,0.00,bonf,1428.771,1.19
9,Presentation,MediaType,Hybrid,Legacy,False,True,4.06,64.96,two-sided,0.00,0.00,bonf,173.068,0.97


In [22]:
# # Define the IV and DV variables
# IV = 'Platform'
# DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
# DF = dfv_ColNames
# # Run the ANOVA
# for i in DV:
#     anova_result = pg.anova(data=DF, dv=i, between=IV)
#     globals()[f"ANOVA_{i}"] = anova_result
#     globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
#     pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
#     globals()[f"Pairwise_{i}"] = pairwise_comparisons
#     globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
#     mean_table = DF.groupby(IV)[i].mean().reset_index()
#     std_table = DF.groupby(IV)[i].std().reset_index()
#     mean_std_table = pd.merge(mean_table, std_table, on=IV)
#     mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
#     mean_std_table = mean_std_table.rename(columns={i: 'Value'})
#     globals()[f"mean_std_table_{i}"] = mean_std_table
#     globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
# merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
# merged_anova.reset_index(drop=True, inplace=True)
# merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
# merged_pairwise.reset_index(drop=True, inplace=True)
# merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
# merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

In [23]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,MediaType,2,97,10.93,0.00,0.18
1,Comments,MediaType,2,97,2.69,0.07,0.05
2,MediaIntegrity,MediaType,2,97,12.48,0.00,0.20
3,Presentation,MediaType,2,97,60.28,0.00,0.55


In [24]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,MediaType,Hybrid,Legacy,False,True,0.25,64.33,two-sided,0.80,1.00,bonf,0.257,0.06
1,Likes,MediaType,Hybrid,SMonly,False,True,-3.23,32.51,two-sided,0.00,0.01,bonf,17.872,-0.83
2,Likes,MediaType,Legacy,SMonly,False,True,-3.28,32.62,two-sided,0.00,0.01,bonf,19.693,-0.80
3,Comments,MediaType,Hybrid,Legacy,False,True,-1.28,45.81,two-sided,0.21,0.62,bonf,0.502,-0.33
4,Comments,MediaType,Hybrid,SMonly,False,True,-2.28,45.42,two-sided,0.03,0.08,bonf,2.173,-0.57
5,Comments,MediaType,Legacy,SMonly,False,True,-0.92,59.61,two-sided,0.36,1.00,bonf,0.37,-0.23
6,MediaIntegrity,MediaType,Hybrid,Legacy,False,True,-1.50,63.98,two-sided,0.14,0.42,bonf,0.645,-0.36
7,MediaIntegrity,MediaType,Hybrid,SMonly,False,True,3.50,63.10,two-sided,0.00,0.00,bonf,35.914,0.84
8,MediaIntegrity,MediaType,Legacy,SMonly,False,True,4.77,59.38,two-sided,0.00,0.00,bonf,1428.771,1.19
9,Presentation,MediaType,Hybrid,Legacy,False,True,4.06,64.96,two-sided,0.00,0.00,bonf,173.068,0.97


In [25]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Hybrid,5783.03,8603.68
1,Likes,Legacy,5281.07,7947.59
2,Likes,SMonly,35076.97,50672.91
3,Comments,Hybrid,237.92,528.39
4,Comments,Legacy,466.07,856.32
5,Comments,SMonly,682.12,991.54
6,MediaIntegrity,Hybrid,3.55,0.80
7,MediaIntegrity,Legacy,3.83,0.75
8,MediaIntegrity,SMonly,2.84,0.88
9,Presentation,Hybrid,2.95,0.84


In [26]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,CommLikes,MediaType,2,992,3.30,0.04,0.01
1,CharacterNum,MediaType,2,988,1.38,0.25,0.00
2,CommLength,MediaType,2,993,1.33,0.26,0.00
3,Sentiment,MediaType,2,976,16.38,0.00,0.03
4,Focus,MediaType,2,991,1.02,0.36,0.00
5,Alignment,MediaType,2,537,16.72,0.00,0.06


In [27]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,CommLikes,MediaType,Hybrid,Legacy,False,True,-1.94,465.24,two-sided,0.05,0.16,bonf,0.65,-0.15
1,CommLikes,MediaType,Hybrid,SMonly,False,True,-3.40,639.80,two-sided,0.00,0.00,bonf,26.011,-0.19
2,CommLikes,MediaType,Legacy,SMonly,False,True,-1.78,804.71,two-sided,0.08,0.23,bonf,0.379,-0.11
3,CharacterNum,MediaType,Hybrid,Legacy,False,True,1.65,308.92,two-sided,0.10,0.30,bonf,0.393,0.16
4,CharacterNum,MediaType,Hybrid,SMonly,False,True,0.51,338.26,two-sided,0.61,1.00,bonf,0.111,0.04
5,CharacterNum,MediaType,Legacy,SMonly,False,True,-1.45,788.10,two-sided,0.15,0.45,bonf,0.224,-0.10
6,CommLength,MediaType,Hybrid,Legacy,False,True,-0.06,329.40,two-sided,0.95,1.00,bonf,0.105,-0.01
7,CommLength,MediaType,Hybrid,SMonly,False,True,-1.17,282.62,two-sided,0.24,0.73,bonf,0.19,-0.11
8,CommLength,MediaType,Legacy,SMonly,False,True,-1.44,671.35,two-sided,0.15,0.45,bonf,0.221,-0.10
9,Sentiment,MediaType,Hybrid,Legacy,False,True,-4.91,366.80,two-sided,0.00,0.00,bonf,1.002e+04,-0.46


In [28]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,CommLikes,Hybrid,94.49,236.07
1,CommLikes,Legacy,162.07,525.87
2,CommLikes,SMonly,257.18,1002.81
3,CharacterNum,Hybrid,96.84,97.09
4,CharacterNum,Legacy,82.30,83.45
5,CharacterNum,SMonly,92.31,113.75
6,CommLength,Hybrid,2.21,0.82
7,CommLength,Legacy,2.22,0.75
8,CommLength,SMonly,2.30,0.77
9,Sentiment,Hybrid,2.34,1.05


In [29]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,CommLikes,Publisher,5,989,11.49,0.00,0.05
1,CharacterNum,Publisher,5,985,1.12,0.35,0.01
2,CommLength,Publisher,5,990,0.70,0.63,0.00
3,Sentiment,Publisher,5,973,13.37,0.00,0.06
4,Focus,Publisher,5,988,3.91,0.00,0.02
5,Alignment,Publisher,5,534,8.56,0.00,0.07
